In [17]:
from DataParser import DataParser
from DataProcessing import DataProcessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sys


# Data

In [2]:
df = DataProcessing().Dataset;




/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: DtypeWarning: Columns (3,14) have mixed types. Specify dtype option on import or set low_memory=False.
  if __name__ == '__main__':





# Feature Engineering

### loan_amnt 
* loan_amnt: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.

In [3]:
num = 0
for i in df.columns:
    print( str(num) + " : " + str(i))
    num = num + 1
    
def plot_feature(feature):
    feature_label_0 = df.loc[df['label_month_0'] == 0][feature]
    feature_label_1 = df.loc[df['label_month_0'] == 1][feature]

    sns.distplot(feature_label_0, color='r')
    sns.distplot(feature_label_1, color='g')
    plt.ylabel("% of data")
    plt.legend()
    plt.figure()
#     sns.regplot(feature_label_1,feature_label_1.unique, color='g')



0 : fico
1 : dt_first_pi
2 : flag_fthb
3 : dt_matr
4 : cd_msa
5 : mi_pct
6 : cnt_units
7 : occpy_sts
8 : cltv
9 : dti
10 : orig_upb
11 : ltv
12 : int_rt
13 : channel
14 : ppmt_pnlty
15 : prod_type
16 : st
17 : prop_type
18 : zipcode
19 : id_loan
20 : loan_purpose
21 : orig_loan_term
22 : cnt_borr
23 : svcg_cycle
24 : current_upb
25 : delq_sts
26 : loan_age
27 : mths_remng
28 : cd_zero_bal
29 : dt_zero_bal
30 : current_int_rt
31 : non_int_brng_upb
32 : dt_lst_pi
33 : mi_recoveries
34 : net_sale_proceeds
35 : non_mi_recoveries
36 : expenses
37 : legal_costs
38 : maint_pres_costs
39 : taxes_ins_costs
40 : misc_costs
41 : actual_loss
42 : modcost
43 : label_month_0


### Feature Ranking


In [4]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier

# Build a classification task using 3 informative features
X = df
y = df['label_month_0']

X = X.drop('label_month_0', 1)

# Build a forest and compute the feature importances
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)

forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)



ValueError: could not convert string to float: C

In [ ]:
to = 20

indices_top = indices[::-1][:to][::-1]

# Plot the feature importances of the forest
plt.figure(figsize=(6, 15))
plt.title("Feature importances")
plt.barh(range(len(indices_top)), importances[indices_top],
       color="r", xerr=std[indices_top], align="center", height=0.5)
# If you want to define your own labels,
# change indices to a list of labels on the following line.
column_names_ordered_importance = X.columns[indices_top]

plt.yticks(range(len(indices_top)), column_names_ordered_importance)
plt.ylim([-1, len(indices_top)])
plt.show()



In [ ]:
indices_best_to_worst = indices[::-1] 
indices_best = indices_best_to_worst[:10]

for i in indices_best:
    plot_feature(X.columns[i])
    


# plot_feature('current_upb')
# plot_feature('svcg_cycle')
# plot_feature('loan_age')

In [ ]:
import datetime
import platform
import random

def format_dataset(df, target_class):
    assert isinstance(df, pd.DataFrame)
    labels = df[target_class].values
#     if Globals.GERMAN_DATA:
#         labels = labels - 1
    # one hot a list of integers
    one_hot_labels = np.eye(np.max(labels) + 1)[labels]
    # remove first column - full of 0's
    # if Globals.GERMAN_DATA:
    #     one_hot_labels = np.delete(one_hot_labels, np.s_[0:1], axis=1)
    df = df.drop([target_class], axis=1)
    data = df.values
    # Standardise features
    # data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
    # data = (data - np.min(data, axis=0)) / (np.max(data, axis=0) - np.min(data, axis=0))
    
    data = pd.DataFrame(data)
    
    for column in data: 
        # if not one-hot-encoded
        if 1 != (data[column].max() - data[column].min()):
#             data[column] = (data[column] - data[column].mean()) / data[column].std()
            data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())
    
    # Remove nan values     
    data = data.fillna(0)

    return (np.array(data),one_hot_labels)

def create_train_validation_datasets(df, target_class):
    assert isinstance(df, pd.DataFrame)
    training_set_size = 0.8 * len(df)
    random.seed(len(df) + datetime.datetime.now().day + 1)
    # print(len(df) + datetime.datetime.now().day)
    # random.seed(len(df))
    sudo_ran_sample = [random.randint(1, len(df)) for i in range(1, len(df))]
    msk = np.array(sudo_ran_sample) < training_set_size
    (train_data, train_labels) = format_dataset(df.iloc[msk], target_class)
#     if self.UPSAMPLING:
#         (train_data, train_labels) = self.upsample_training_data(train_data, train_labels)

#     if self.UPSAMPLING_ALL:
#         (train_data, train_labels) = self.upsample_all(train_data, train_labels)
#     # Shuffle dataset?????

    (validation_data, validation_labels) = format_dataset(df.iloc[~msk], target_class)

    # Shuffle dataset?????

#     self.print_class_ratios(train_labels, "Training Class ratios BEFORE")
#     self.print_class_ratios(validation_labels, "Validation Class ratios BEFORE")

    return ((train_data, train_labels) , (validation_data, validation_labels))

In [ ]:

(train, validation) = create_train_validation_datasets(df, 'label_month_0')


In [ ]:
df_train = pd.DataFrame(validation[0])
df_fresh = pd.DataFrame(df)
# print pd.DataFrame(validation[0]).isnull().sum()
# print pd.DataFrame(df).isnull().sum()

print(df_train.max() - df_train.min())
print(df_fresh.max() - df_fresh.min())



In [ ]:
df_fresh.columns = range(0,len(df_fresh.columns))

print df_fresh.loc[:, 40:42]

sns.distplot(df_train[42], color='r')
plt.figure()
sns.distplot(df_fresh[42], color='g')

In [ ]:
df = DataProcessing().Dataset;


In [ ]:
print df['zipcode']

In [ ]:
def plot_feature(feature):
    feature_label_0 = df.loc[df[DataProcessing().TargetClass] == 0][feature]
    feature_label_1 = df.loc[df[DataProcessing().TargetClass] == 1][feature]
    bins=np.histogram(np.hstack((feature_label_0,feature_label_1)), bins=40)[1] #get the bin edges
    
    feature_label_1_5x = pd.concat(([feature_label_1]))

    fig, ax = plt.subplots(figsize=(12,6),sharey=True)
    sns.despine(left=True)
    ax = sns.distplot(feature_label_0, bins=bins, color='g', label="Good Loan", kde=False)
    ax = sns.distplot(feature_label_1_5x, bins=bins, color='r', label="Bad Loan x 10", kde=False)
    ax.set_xlabel(feature)
    ax.set_ylabel('Number of samples')
    ax.legend()
    plt.show()

In [ ]:
def filter_data_monthly(data):
    columns__monthly = ['svcg_cycle', 'current_upb', 'delq_sts', 'loan_age', 'mths_remng', 'repch_flag', 'flag_mod',
    'cd_zero_bal', 'dt_zero_bal', 'current_int_rt', 'non_int_brng_upb', 'dt_lst_pi', 'mi_recoveries',
    'net_sale_proceeds', 'non_mi_recoveries', 'expenses', 'legal_costs', 'maint_pres_costs',
    'taxes_ins_costs', 'misc_costs', 'actual_loss', 'modcost']
    
    
    for column in data.columns:
        for column_monthly in columns__monthly:
            if column_monthly in column:
#                 print df[column].unique()
                plot_feature(column)
    
df = df.fillna(-255)
filter_data_monthly(df)

In [5]:
# print df.loc[df['delq_sts'] == 5]['delq_sts']
print df['svcg_cycle']


0         200208.0
1         200209.0
2         200210.0
3         200211.0
4         200212.0
5         200301.0
6         200301.0
7         200302.0
8         200303.0
9         200304.0
10        200305.0
11        200306.0
12        200307.0
13        200308.0
14        200309.0
15        200310.0
16        200311.0
17        200312.0
18        200401.0
19        200402.0
20        200403.0
21        200404.0
22        200405.0
23        200406.0
24        200407.0
25        200408.0
26        200409.0
27        200410.0
28        200411.0
29        200412.0
30        200501.0
31        200502.0
32        200503.0
33        200504.0
34        200505.0
35        200506.0
36        200507.0
37        200508.0
38        200509.0
39        200510.0
40        200511.0
41        200512.0
42        200601.0
43        200602.0
44        200603.0
45        200604.0
46        200605.0
47        200606.0
48        200607.0
49        200608.0
50        200609.0
51        200610.0
52        20

In [18]:
nat_int_rt = DataParser().national_mortgage_rate()

df_1 = pd.merge(nat_int_rt, df, on='svcg_cycle', how='right')

print df_1[['svcg_cycle', 'nat_int_rt']]


        svcg_cycle  nat_int_rt
0           199902        6.81
1           199902        6.81
2           199902        6.81
3           199902        6.81
4           199902        6.81
5           199902        6.81
6           199902        6.81
7           199902        6.81
8           199902        6.81
9           199902        6.81
10          199902        6.81
11          199902        6.81
12          199902        6.81
13          199902        6.81
14          199902        6.81
15          199902        6.81
16          199902        6.81
17          199902        6.81
18          199902        6.81
19          199902        6.81
20          199902        6.81
21          199902        6.81
22          199902        6.81
23          199902        6.81
24          199902        6.81
25          199902        6.81
26          199902        6.81
27          199902        6.81
28          199902        6.81
29          199902        6.81
30          199902        6.81
31      

In [34]:
df_1_high = df.loc[df['id_loan'] == 'F199Q1159323']
df_1_low = df.loc[df['id_loan'] == 'F199Q1059450']
df_main = pd.concat((df_1_high, df_1_low))

# df_main = df[:200]

# Introduce new dataset - national mortgage rate
df_nat_int_rt = DataParser().national_mortgage_rate()

df_main = pd.merge(df_nat_int_rt, df_main, on='svcg_cycle', how='right')

# Initialise new columns
df_main.loc[:, 'time_since_origin'] = 1

df_main.loc[:, 'pct_change'] = 0
df_main.loc[:, 'crt_minus_nat_int_rt'] = 0
df_main.loc[:, 'occr_crt_less_than_nat_int_rate'] = 0


df_main.loc[:, 'occr_curr_12_mon'] = 0
df_main.loc[:, 'occr_curr'] = 0

df_main.loc[:, 'occr_30dd_12_mon'] = 0
df_main.loc[:, 'occr_30dd'] = 0

df_main.loc[:, 'occr_60dd_12_mon'] = 0
df_main.loc[:, 'occr_60dd'] = 0

df_main.loc[:, 'occr_90dd_12_mon'] = 0
df_main.loc[:, 'occr_90dd'] = 0

df_main.loc[:, 'occr_foreclosed_12_mon'] = 0
df_main.loc[:, 'occr_foreclosed'] = 0

for id_loan in df_main['id_loan'].unique():
    df_1 = df_main.loc[df_main['id_loan'] == id_loan]

    # Time since origin
    df_1['time_since_origin'] = df_1['time_since_origin'].cumsum()

    # ----------------------------------------------------------------------     

    # Number of occurrences of current
    query = df_1.query("delq_sts == 0").index
    df_1.loc[query, 'occr_curr'] = 1
    df_1['occr_curr'] = df_1['occr_curr'].cumsum()

    # Number of occurrences of current in last 12 months
    loan_duration = df_1['id_loan'].value_counts().values
    if loan_duration[0] >= 12:
        df_1.loc[:, 'occr_curr_12_mon'] = df_1['occr_curr'] - df_1['occr_curr'].shift(12)
        df_1['occr_curr_12_mon'][:12] = df_1['occr_curr'][:12]
    else:
        df_1['occr_curr_12_mon'] = df_1['occr_curr']

    # ----------------------------------------------------------------------     

    # Number of occurrences of 30 days delinquent
    query = df_1.query("delq_sts == 1").index
    df_1.loc[query, 'occr_30dd'] = 1
    df_1['occr_30dd'] = df_1['occr_30dd'].cumsum()

    # Number of occurrences of 30 days delinquent in last 12 months
    loan_duration = df_1['id_loan'].value_counts().values
    if loan_duration[0] >= 12:
        df_1.loc[:, 'occr_30dd_12_mon'] = df_1['occr_30dd'] - df_1['occr_30dd'].shift(12)
        df_1['occr_30dd_12_mon'][:12] = df_1['occr_30dd'][:12]
    else:
        df_1['occr_30dd_12_mon'] = df_1['occr_30dd']

    # ----------------------------------------------------------------------     

    # Number of occurrences of 60 days delinquent
    query = df_1.query("delq_sts == 2").index
    df_1.loc[query, 'occr_60dd'] = 1
    df_1['occr_60dd'] = df_1['occr_60dd'].cumsum()

    # Number of occurrences of 60 days delinquent in last 12 months
    loan_duration = df_1['id_loan'].value_counts().values
    if loan_duration[0] >= 12:
        df_1.loc[:, 'occr_60dd_12_mon'] = df_1['occr_60dd'] - df_1['occr_60dd'].shift(12)
        df_1['occr_60dd_12_mon'][:12] = df_1['occr_60dd'][:12]
    else:
        df_1['occr_60dd_12_mon'] = df_1['occr_60dd']

    # ----------------------------------------------------------------------     

    # Number of occurrences of 90+ days delinquent
    query = df_1.query("delq_sts > 2").index
    df_1.loc[query, 'occr_90dd'] = 1
    df_1['occr_90dd'] = df_1['occr_90dd'].cumsum()

    # Number of occurrences of 90+ days delinquent in last 12 months
    loan_duration = df_1['id_loan'].value_counts().values
    if loan_duration[0] >= 12:
        df_1.loc[:, 'occr_90dd_12_mon'] = df_1['occr_90dd'] - df_1['occr_90dd'].shift(12)
        df_1['occr_90dd_12_mon'][:12] = df_1['occr_90dd'][:12]
    else:
        df_1['occr_90dd_12_mon'] = df_1['occr_90dd']

    # ----------------------------------------------------------------------     

    # Number of occurrences of Foreclosed
    query = df_1.query("label_month_0 == 4").index
    df_1.loc[query, 'occr_foreclosed'] = 1
    df_1['occr_foreclosed'] = df_1['occr_foreclosed'].cumsum()

    # Number of occurrences of Foreclosed in last 12 months
    loan_duration = df_1['id_loan'].value_counts().values
    if loan_duration[0] >= 12:
        df_1.loc[:, 'occr_foreclosed_12_mon'] = df_1['occr_foreclosed'] - df_1['occr_foreclosed'].shift(12)
        df_1['occr_foreclosed_12_mon'][:12] = df_1['occr_foreclosed'][:12]
    else:
        df_1['occr_foreclosed_12_mon'] = df_1['occr_foreclosed']

    # ----------------------------------------------------------------------    

    # percentage change between last balance and current balance 
    df_1['pct_change'] = (df_1['current_upb'] - df_1['current_upb'].shift(1)) / df_1['current_upb'].shift(1)
    df_1.loc[df_1['pct_change'] == -1.0, 'pct_change'] = 0
    df_1 = df_1.fillna(0)

    # ----------------------------------------------------------------------    

    # Current interest rate − national mortgage rate
    df_1['crt_minus_nat_int_rt'] = df_1['current_int_rt'] - df_1['nat_int_rt']
    
    # ----------------------------------------------------------------------    

    # Number of months the mortgage’s interest been less than the national mortgage rate
    query = df_1.query("crt_minus_nat_int_rt < 0").index
    df_1.loc[query, 'occr_crt_less_than_nat_int_rate'] = 1
    df_1['occr_crt_less_than_nat_int_rate'] = df_1['occr_crt_less_than_nat_int_rate'].cumsum()

    # ----------------------------------------------------------------------    
    # ----------------------------------------------------------------------    

    # Set main dataframe to local values for current loan     
    df_main.loc[df_main['id_loan'] == id_loan] = df_1.loc[df_1['id_loan'] == id_loan]

print df_main[['time_since_origin', 'crt_minus_nat_int_rt', 'occr_crt_less_than_nat_int_rate']]


     time_since_origin  crt_minus_nat_int_rt  occr_crt_less_than_nat_int_rate
0                    1                -0.045                                1
1                    2                -0.275                                2
2                    1                 0.100                                0
3                    3                -0.675                                3
4                    2                -0.300                                1
5                    4                -0.755                                4
6                    3                -0.380                                2
7                    5                -1.065                                5
8                    4                -0.690                                3
9                    6                -0.945                                6
10                   5                -0.570                                4
11                   7                -0.975                    